In [2]:
import os
import pandas as pd
import numpy as np
import json
import re

# ! pip install openai
# ! pip install python-dotenv
from openai import OpenAI
from dotenv import load_dotenv

##### 법령 추출

In [4]:
# 금융보안원 test 자료 불러오기
test = pd.read_csv("DATA/금융보안원_test.csv")
test.head(2)

,ID,Question
0,TEST_000,금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?\n1 소비자금융...
1,TEST_001,위험 관리 계획 수립 시 고려해야 할 요소로 적절하지 않은 것은?\n1 수행인력\n...


In [5]:
class GetAnswersLLM():
    def __init__(self, llm):
        """llm : 'gpt' or 'solar'"""
        self.llm = llm
        self.system_prompt = (
            "당신은 금융 전문가입니다. 사용자의 질문에 대해 정확하고 책임감 있게 답변하세요.\n"
            "만약 질문을 해결하는 데 특정 법률(예: 개인정보 보호법 등)이 필요하다면, 해당 법률명과 관련 조항을 명시하세요.\n"
            "응답은 반드시 다음 JSON 형식을 따르세요:\n"
            "{\n"
            '  "Answer": "...",\n'
            '  "Laws": "...",\n'
            '  "Laws_YN": "..."\n'
            "}"
            "JSON 형식의 각 항목에 대한 설명은 다음과 같습니다:\n"
            "Answer 란에는 정답을 입력합니다.\n"
            "Laws 란에는 정답을 찾는 과정에서 사용된 법률 및 조항을 명시합니다. (제 O조 제 O항)\n"
            "Laws_YN 란에는 정답을 찾는 과정에서 법률이 사용되었는지 여부를 나타냅니다. (Yes, No 로 명시) \n"
            "응답은 반드시 위 JSON 형식 그대로만 출력해야 합니다. JSON 외의 추가 텍스트나 설명은 절대 포함하지 마세요."
        )
    
    # api key 불러오기
    def get_api(self):
        load_dotenv()
        if self.llm == "solar":
            SOLAR_API_KEY = os.environ.get("SOLAR_API_KEY")
            if not SOLAR_API_KEY:
                raise EnvironmentError("SOLAR_API_KEY not found in .env")
            return SOLAR_API_KEY

        if self.llm == "gpt":
            OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
            if not OPENAI_API_KEY:
                    raise EnvironmentError("OPENAI_API_KEY not found in .env")
            return OPENAI_API_KEY
        
    # client 설정
    def get_client(self):
        if self.llm == "solar":
            api = self.get_api()
            if not api:
                raise EnvironmentError("No API Found")
            client = OpenAI(api_key = api,
                            base_url = "https://api.upstage.ai/v1")
            return client 
        if self.llm == "gpt":
            api = self.get_api()
            client = OpenAI(api_key = api)
            return client

    # 응답 생성
    def get_rules(self, question):

        client = self.get_client()

        if self.llm == "solar":
            model_name = "solar-pro2"
        if self.llm == "gpt":
            model_name = "gpt-4o-mini"

        response = client.chat.completions.create(
            model = model_name,
            messages = [
                {"role" : "system", "content" : self.system_prompt},
                {"role" : "user", "content" : question}
            ],
            temperature = 0.3
        )
        response = response.choices[0].message.content
        return response 
    
    def update_data(self, data):
        for idx in range(len(data)):
            question = data.loc[idx, 'Question']
            response = self.get_rules(question)

            try:
                parsed = json.loads(response)
            except json.JSONDecodeError:
                parsed = {"Answer": None, "Laws": None, "Laws_YN": None}

            data.loc[idx, "Answer"] = parsed.get("Answer")
            data.loc[idx, "Laws"] = parsed.get("Laws")
            data.loc[idx, "Laws_YN"] = parsed.get("Laws_YN")
            
        return data


In [6]:
# SOLAR 사용
llm = GetAnswersLLM(llm = "solar")
question = test['Question'][0]
response = llm.get_rules(question)
response


'{\n  "Answer": "1 소비자금융업, 5 보험중개업",\n  "Laws": "자본시장과 금융투자업에 관한 법률 제6조 제1항, 제3항",\n  "Laws_YN": "Yes"\n}'

In [7]:
llm = GetAnswersLLM(llm = "solar")
test_updated = llm.update_data(test)
test_updated

,ID,Question,Answer,Laws,Laws_YN
0,TEST_000,금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?\n1 소비자금융...,"1 소비자금융업, 5 보험중개업","자본시장과 금융투자업에 관한 법률 제6조 제1항, 제2항, 제3항, 제4항, 제5항...",Yes
1,TEST_001,위험 관리 계획 수립 시 고려해야 할 요소로 적절하지 않은 것은?\n1 수행인력\n...,4 대상,,No
2,TEST_002,관리체계 수립 및 운영'의 '정책 수립' 단계에서 가장 중요한 요소는 무엇인가?\n...,2 경영진의 참여,개인정보 보호법 제32조(개인정보 보호책임자의 지정) 및 제28조(안전성 확보 조치...,Yes
3,TEST_003,재해 복구 계획 수립 시 고려해야 할 요소로 옳지 않은 것은?\n1 복구 절차 수립...,3 개인정보 파기 절차,개인정보 보호법 제21조(개인정보의 파기),Yes
4,TEST_004,트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표...,트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표...,,No
...,...,...,...,...,...
510,TEST_510,"""정보보호최고책임자""의 임명에 관한 설명으로 옳지 않은 것은?\n1 정보보호최고책임...",5,"전자금융거래법 제21조의2(정보보호최고책임자의 지정 등) 제1항, 제3항, 제4항,...",Yes
511,TEST_511,IPv6 주소 체계의 주요 특징으로 옳지 않은 것은?\n1 NAT 필요성 감소\n2...,None,None,None
512,TEST_512,하이브리드 위협에 대한 설명으로 가장 적절한 것은?\n1 사이버 공간에서만 발생하는...,3 온·오프라인 연계를 통해 다양한 채널을 이용하는 위협,,No
513,TEST_513,전자금융거래법의 주요 목적 중 하나는 무엇인가?\n1 전자금융거래의 비대면성 강화\...,4 전자금융거래의 안전성과 신뢰성 확보,전자금융거래법 제1조(목적) 이 법은 전자금융거래의 안전성과 신뢰성을 확보하고 전자...,Yes


In [13]:
test_updated['Laws_YN'].value_counts()

Laws_YN
Yes    391
No     105
Name: count, dtype: int64

In [12]:
test_updated.isna().sum()

ID           0
Question     0
Answer      19
Laws        19
Laws_YN     19
dtype: int64

In [ ]:
test_updated.to_pickle("C:/Users/SAMSUNG/Desktop/대학원/랩미팅/2025.09.10_국가법령정보센터/DATA/test_updated.pickle")

##### 법령 정리

In [ ]:
### 필요한 법령 가져오기
test_updated = pd.read_pickle("C:/Users/SAMSUNG/Desktop/대학원/랩미팅/2025.09.10_국가법령정보센터/DATA/test_updated.pickle")

# "~법", "~법률" 부분만 가져오기
def get_law(laws_org):
    if not isinstance(laws_org, str):
        return None
    pattern = re.compile(r"(?:「)?([가-힣\s]+법(?:률)?)(?:」)?")
    matched = pattern.search(laws_org)
    return matched.group(1) if matched else None

test_updated["Only_Law"] = test_updated["Laws"].map(get_law)

In [ ]:
sorted(test_updated["Only_Law"].dropna().unique())

##### 법령 불러오기

In [15]:
import requests, urllib.parse, os, json, re
import pandas as pd
import numpy as np
from pathlib import Path

In [23]:
### 최종 
laws_to_fetch = ['개인정보 보호법', '국가보안법', '국가정보화 기본법', 
                 '금융소비자 보호에 관한 법률', '금융위원회의 설치 등에 관한 법률',
                 '금융회사의 정보보호에 관한 법률', '금융회사의 지배구조에 관한 법률',
                 '민법', '보험업법', '상호저축은행법', '신용정보의 이용 및 보호에 관한 법률',
                 '신용협동조합법', '여신전문금융업법', '은행법', 
                 '자본시장과 금융투자업에 관한 법률', '전기통신사업법', '전자금융거래법',
                 '전자문서 및 전자거래 기본법', '전자서명법', '전자정부법', '정보통신기반 보호법',
                 '정보통신망 이용촉진 및 정보보호 등에 관한 법률', '주민등록법',
                 '채권추심규제법', '청소년 보호법', '클라우드컴퓨팅 발전 및 이용자 보호에 관한 법률',
                 '특정 금융거래정보의 보고 및 이용 등에 관한 법률', '한국은행법',
                 '한국자산관리공사 설립 등에 관한 법률', '형법',
                 # 추가
                 '주택임대차보호법', '부동산 실권리자명의 등기에 관한 법률', '부동산 거래신고 등에 관한 법률', 
                 '소득세법', '상속세 및 증여세법', '조세특례제한법', '근로기준법', '근로자퇴직급여 보장법', 
                 '예금자보호법', '대부업 등의 등록 및 금융이용자 보호에 관한 법률', '이자제한법', 
                 '약관의 규제에 관한 법률', '상법', '채권의 공정한 추심에 관한 법률', 
                 '국민건강보험법', '산업재해보상보험법', '고용보험법', '국민연금법', '자동차손해배상 보장법', 
                 '교통사고처리 특례법', '화재로 인한 재해보상과 보험가입에 관한 법률', '재난 및 안전관리 기본법', 
                 '우체국예금ㆍ보험에 관한 법률', '새마을금고법', '농업협동조합법']

In [24]:
class FetchLaws():
    def __init__(self):
        self.API = "jhpark0256"
        self.PAGE = "http://www.law.go.kr/DRF"

    def fetch_law_id(self, i, law_name):
        """법 이름 하나 받아서 법 정보 출력"""
        url = f"{self.PAGE}/lawSearch.do?OC={self.API}&target=law&type=JSON&query={urllib.parse.quote(law_name)}&display=10&search=1"
        contents = requests.get(url, timeout = 15)
        contents.raise_for_status() # 오류 나면 예외 발생
        contents_json = contents.json() # 응답 -> json 형태 
        items = contents_json.get("LawSearch").get("law", []) # 'law' 부분 꺼내오기
            
        # 항목 한 개 매칭 -> list로 감싸기
        if isinstance(items, dict):
            items = [items]

        if not items:
            print(f"❌ [{i}. {law_name}] 매칭 실패..")
            return None
        
        for item in items:
            if item.get('법령명한글') == law_name:
                print(f"✅ [{i}. {law_name}] 매칭 완료 !")
                return {'ID': item.get('법령ID'), 'MST': item.get('법령일련번호'), 'law': item.get('법령명한글')}
        print(f"🔁 [{i}. {law_name}] -> [{items[0].get('법령명한글')}](으)로 대체..")
        return{'ID': items[0].get('법령ID'), 'MST': items[0].get('법령일련번호'), 'law': items[0].get('법령명한글')}
    
    def fetch_law_text(self, law_id):
        """법 ID 받아서 법 본문 출력"""
        url = f"{self.PAGE}/lawService.do"
        params = {"OC" : self.API, "target" : "law", "ID" : law_id, "type" : "JSON"}
        contents = requests.get(url, params = params, timeout = 20)
        contents.raise_for_status()
        data = contents.json()
        return data

In [8]:
os.getcwd()

'c:\\Users\\SAMSUNG\\Desktop\\Grad_School\\RAG_LAW\\ipynb_files'

In [ ]:
fetchclass = FetchLaws()

for i, law_name in enumerate(laws_to_fetch, 1):
    meta = fetchclass.fetch_law_id(i, law_name)
    if meta is None:
        continue
    file_path = Path(f'../DATA/Raws/Laws/{meta.get("law")}.json')
    law_id = meta.get("ID")
    contents = fetchclass.fetch_law_text(law_id)
    with open(file_path, 'w', encoding = 'utf-8') as f:
        json.dump(contents, f, ensure_ascii = False, indent = 4)

✅ [1. 개인정보 보호법] 매칭 완료 !
✅ [2. 국가보안법] 매칭 완료 !
🔁 [3. 국가정보화 기본법] -> [지능정보화 기본법](으)로 대체..
✅ [4. 금융소비자 보호에 관한 법률] 매칭 완료 !
✅ [5. 금융위원회의 설치 등에 관한 법률] 매칭 완료 !
❌ [6. 금융회사의 정보보호에 관한 법률] 매칭 실패..
✅ [7. 금융회사의 지배구조에 관한 법률] 매칭 완료 !
✅ [8. 민법] 매칭 완료 !
✅ [9. 보험업법] 매칭 완료 !
✅ [10. 상호저축은행법] 매칭 완료 !
✅ [11. 신용정보의 이용 및 보호에 관한 법률] 매칭 완료 !
✅ [12. 신용협동조합법] 매칭 완료 !
✅ [13. 여신전문금융업법] 매칭 완료 !
✅ [14. 은행법] 매칭 완료 !
✅ [15. 자본시장과 금융투자업에 관한 법률] 매칭 완료 !
✅ [16. 전기통신사업법] 매칭 완료 !
✅ [17. 전자금융거래법] 매칭 완료 !
✅ [18. 전자문서 및 전자거래 기본법] 매칭 완료 !
✅ [19. 전자서명법] 매칭 완료 !
✅ [20. 전자정부법] 매칭 완료 !
✅ [21. 정보통신기반 보호법] 매칭 완료 !
✅ [22. 정보통신망 이용촉진 및 정보보호 등에 관한 법률] 매칭 완료 !
✅ [23. 주민등록법] 매칭 완료 !
❌ [24. 채권추심규제법] 매칭 실패..
✅ [25. 청소년 보호법] 매칭 완료 !
✅ [26. 클라우드컴퓨팅 발전 및 이용자 보호에 관한 법률] 매칭 완료 !
✅ [27. 특정 금융거래정보의 보고 및 이용 등에 관한 법률] 매칭 완료 !
✅ [28. 한국은행법] 매칭 완료 !
✅ [29. 한국자산관리공사 설립 등에 관한 법률] 매칭 완료 !
✅ [30. 형법] 매칭 완료 !
✅ [31. 주택임대차보호법] 매칭 완료 !
✅ [32. 부동산 실권리자명의 등기에 관한 법률] 매칭 완료 !
✅ [33. 부동산 거래신고 등에 관한 법률] 매칭 완료 !
✅ [34. 소득세법] 매칭 완료 !
✅ [35. 상속세 및 증여세법] 매칭 완료 !
✅ [36. 